In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image

from tqdm import tqdm, trange
from difflib import SequenceMatcher

plt.style.use('ggplot')

In [ ]:
annot = pd.read_parquet('../input/textocr-text-extraction-from-images-dataset/annot.parquet')
img_fns = glob('../input/textocr-text-extraction-from-images-dataset/train_val_images/train_images/*')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(plt.imread(img_fns[11]))
ax.axis('off')
plt.show()

In [ ]:
len(img_fns)

In [ ]:
import pytesseract

# Example call
print(pytesseract.image_to_string(img_fns[13], lang='eng'))

# Pytesseract

In [ ]:
####Loop through each image to extract text and save in df. Meanwhile calculate run time
import pytesseract
from pytesseract import Output
from tqdm import tqdm, trange
from difflib import SequenceMatcher


#create an empty dataframe
df=pd.DataFrame(columns = ['image_id', 'text'])

#loop through each image and append results
for i in trange(1000):
    image_id = img_fns[i].split('/')[-1].rstrip('.jpg')
    data=pytesseract.image_to_data(img_fns[i],output_type=Output.DATAFRAME )
    data['image_id']=image_id
    df=df.append(data[['image_id','text']],ignore_index=True)
#df.to_csv(r'\Users\magzhang\Documents\ray.csv', index=False)
#remove NaN and missing values from df
df1=df[(df.text!=' ')].dropna().astype(str)

#concatenate text field by groupby image_id
df2=df1.sort_values('text').groupby(['image_id'], as_index=False).agg({'text': ' '.join})
annot_agg=annot[['image_id','utf8_string']].sort_values('utf8_string').groupby(['image_id'], as_index=False).agg({'utf8_string': ' '.join})

final_p=annot_agg.merge(df2, how='inner', left_on='image_id',right_on='image_id')

def apply_sm(df):
    if not df[['utf8_string', 'text']].isnull().any():
        return SequenceMatcher(None, df['utf8_string'], df['text']).ratio()
    return 0.0
final_p['ratio']=final_p.apply(apply_sm,axis=1)
final_p["ratio"].mean()

# Modified OCR

In [ ]:
#define function to identify if handwritting
import cv2
def is_handwritten(im):
# Find contours in the edged image   
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 30, 150)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    is_handwritten = False
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)
        #print(aspect_ratio)
        if aspect_ratio < 1:
            return True
        else:
            return False

In [ ]:
im = cv2.imread(img_fns[15])
is_handwritten(im)

In [ ]:
import pytesseract
from pytesseract import Output
from tqdm import tqdm, trange
from difflib import SequenceMatcher
import easyocr
reader = easyocr.Reader(['en'], gpu = True)

In [ ]:
####Loop through each image to extract text and save in df. Meanwhile calculate run time


#create an empty dataframe
df=pd.DataFrame(columns = ['image_id', 'text'])

#loop through each image and append results
for i in trange(1000):
    image_id = img_fns[i].split('/')[-1].rstrip('.jpg')
    im = cv2.imread(img_fns[i])
    if is_handwritten(im):
        data=pytesseract.image_to_data(img_fns[i],output_type=Output.DATAFRAME )
    else:
        results = reader.readtext(img_fns[i])
        data=pd.DataFrame(results, columns=['bbox','text','conf'])
    
    data['image_id']=image_id
    df=df.append(data[['image_id','text']],ignore_index=True)

    
df1=df[(df.text!=' ')].dropna().astype(str)

#concatenate text field by groupby image_id
df2=df1.sort_values('text').groupby(['image_id'], as_index=False).agg({'text': ' '.join})
annot_agg=annot[['image_id','utf8_string']].sort_values('utf8_string').groupby(['image_id'], as_index=False).agg({'utf8_string': ' '.join})

final_p=annot_agg.merge(df2, how='inner', left_on='image_id',right_on='image_id')

def apply_sm(df):
    if not df[['utf8_string', 'text']].isnull().any():
        return SequenceMatcher(None, df['utf8_string'], df['text']).ratio()
    return 0.0
final_p['ratio']=final_p.apply(apply_sm,axis=1)
final_p["ratio"].mean()

# Easy OCR

In [ ]:
import easyocr
reader = easyocr.Reader(['en'], gpu = True)

In [ ]:
#create an empty dataframe
df=pd.DataFrame(columns = ['image_id', 'text'])

#loop through each image and append results
for i in trange(1000):
    image_id = img_fns[i].split('/')[-1].rstrip('.jpg')
    results = reader.readtext(img_fns[i])
    data=pd.DataFrame(results, columns=['bbox','text','conf'])
    data['image_id']=image_id
    df=df.append(data[['image_id','text']],ignore_index=True)


df1=df[(df.text!=' ')].dropna().astype(str)

#concatenate text field by groupby image_id
df2=df1.sort_values('text').groupby(['image_id'], as_index=False).agg({'text': ' '.join})
annot_agg=annot[['image_id','utf8_string']].sort_values('utf8_string').groupby(['image_id'], as_index=False).agg({'utf8_string': ' '.join})

final_p=annot_agg.merge(df2, how='inner', left_on='image_id',right_on='image_id')

def apply_sm(df):
    if not df[['utf8_string', 'text']].isnull().any():
        return SequenceMatcher(None, df['utf8_string'], df['text']).ratio()
    return 0.0
final_p['ratio']=final_p.apply(apply_sm,axis=1)
final_p["ratio"].mean()